In [32]:
import pandas as pd
import json
from datetime import datetime
import os

In [33]:

# Define base directories
base_dir = "../"  # Base directory, relative to the notebook
processed_dir = os.path.join(base_dir, "data", "processed")

# Define file paths 
cleaned_csv_path = os.path.join(processed_dir, "cleaned_stock_data.csv")
cleaned_json_path = os.path.join(processed_dir, "cleaned_stock_data.json")

# Access the cleaned CSV file
df_cleaned_csv = pd.read_csv(cleaned_csv_path)
print("CSV Data Loaded:")
print(df_cleaned_csv.head())

# Access the cleaned JSON file
df_cleaned_json = pd.read_json(cleaned_json_path)
print("\nJSON Data Loaded:")
print(df_cleaned_json.head())


final_merged_path  = os.path.join(processed_dir, "final_merged_stock_data.csv")

CSV Data Loaded:
         Date       AAPL     AMZN      GOOGL         IBM       META  \
0  2015-01-02  24.347170  15.4260  26.381865  100.658279  78.151474   
1  2015-01-05  23.661274  15.1095  25.879185   99.074463  76.896263   
2  2015-01-06  23.663502  14.7645  25.240503   96.937805  75.860222   
3  2015-01-07  23.995316  14.9210  25.166271   96.304253  75.860222   
4  2015-01-08  24.917269  15.0230  25.253954   98.397438  77.882492   

        MSFT       NFLX      NVDA       ORCL  ...     AAPL.5    AMZN.5  \
0  40.152485  49.848572  0.483143  37.863850  ...  212818400  55664000   
1  39.783253  47.311428  0.474983  37.332840  ...  257142000  55484000   
2  39.199337  46.501431  0.460582  36.947441  ...  263188400  70380000   
3  39.697376  46.742859  0.459382  36.956005  ...  160423600  52806000   
4  40.865208  47.779999  0.476663  37.178692  ...  237458000  61768000   

    GOOGL.5    IBM.5    META.5    MSFT.5    NFLX.5     NVDA.5    ORCL.5  \
0  26480000  5779673  18177500  2791

In [34]:

# ✅ Step 1: Load Cleaned Data
df_csv = pd.read_csv(cleaned_csv_path)
df_json = pd.read_json(cleaned_json_path)

In [39]:
# Load JSON
df_json = pd.read_json(cleaned_json_path)

# Print available columns
print("🔍 JSON Columns:", df_json.columns)
print("🔍 JSON Index Name:", df_json.index.name)
print(df_json.head())  # Preview the first few rows

🔍 JSON Columns: Index(['AAPL_Adj Close', 'AAPL_Close', 'AAPL_High', 'AAPL_Low', 'AAPL_Open',
       'AAPL_Volume', 'AMZN_Adj Close', 'AMZN_Close', 'AMZN_High', 'AMZN_Low',
       'AMZN_Open', 'AMZN_Volume', 'GOOGL_Adj Close', 'GOOGL_Close',
       'GOOGL_High', 'GOOGL_Low', 'GOOGL_Open', 'GOOGL_Volume',
       'IBM_Adj Close', 'IBM_Close', 'IBM_High', 'IBM_Low', 'IBM_Open',
       'IBM_Volume', 'META_Adj Close', 'META_Close', 'META_High', 'META_Low',
       'META_Open', 'META_Volume', 'MSFT_Adj Close', 'MSFT_Close', 'MSFT_High',
       'MSFT_Low', 'MSFT_Open', 'MSFT_Volume', 'NFLX_Adj Close', 'NFLX_Close',
       'NFLX_High', 'NFLX_Low', 'NFLX_Open', 'NFLX_Volume', 'NVDA_Adj Close',
       'NVDA_Close', 'NVDA_High', 'NVDA_Low', 'NVDA_Open', 'NVDA_Volume',
       'ORCL_Adj Close', 'ORCL_Close', 'ORCL_High', 'ORCL_Low', 'ORCL_Open',
       'ORCL_Volume', 'TSLA_Adj Close', 'TSLA_Close', 'TSLA_High', 'TSLA_Low',
       'TSLA_Open', 'TSLA_Volume'],
      dtype='object')
🔍 JSON Index Name: N

In [40]:
# ✅ **Fix: Convert JSON Index to "Date" Column**
df_json = df_json.reset_index().rename(columns={"index": "Date"})

# ✅ Step 2: Ensure Proper Date Formatting
df_csv["Date"] = pd.to_datetime(df_csv["Date"], errors="coerce")
df_json["Date"] = pd.to_datetime(df_json["Date"], errors="coerce")

In [41]:
# ✅ Step 3: Align Date Ranges
min_date = min(df_csv["Date"].min(), df_json["Date"].min())
max_date = max(df_csv["Date"].max(), df_json["Date"].max())
date_range = pd.date_range(start=min_date, end=max_date)

In [42]:
df_csv = df_csv.set_index("Date").reindex(date_range).reset_index().rename(columns={"index": "Date"}).ffill()
df_json = df_json.set_index("Date").reindex(date_range).reset_index().rename(columns={"index": "Date"}).ffill()

In [43]:
# ✅ Step 4: Merge Cleaned Data
merged_df = pd.merge(df_csv, df_json, on="Date", how="outer")

In [44]:
# ✅ Step 5: Fill Missing Values
merged_df.ffill(inplace=True)  # Forward fill
merged_df.bfill(inplace=True)  # Backward fill

In [45]:
# ✅ Step 6: Save the Final Cleaned Data
merged_df.to_csv(final_merged_path, index=False)
print(f"✅ Final Merged Stock Data saved at: {final_merged_path}")

✅ Final Merged Stock Data saved at: ../data/processed/final_merged_stock_data.csv
